# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("../output_data/cities.csv")
weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
locations = weather[["Lat","Lng"]].astype(float)
humidity = weather["Humidity"].astype(float)

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,-80))
                                                   
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
temp_pref = (weather["Max Temp"] < 80) & (weather["Max Temp"] > 65)
windspeed_pref = weather["Wind Speed"] <20
cloud_pref = weather["Cloudiness"] <5
humidity_pref = weather["Humidity"] < 60
weather_pref = temp_pref & windspeed_pref & cloud_pref & humidity_pref

pref_weather_df = weather[weather_pref]
pref_weather_df = pref_weather_df.dropna()

pref_weather_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
56,56,pisco,0,PE,1558378764,30,-13.71,-76.20,75.99,11.41
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pref_weather_df
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row ["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    hotel_name = requests.get(base_url, params = params)
    
    hotel_name = hotel_name.json()

    print(json.dumps(hotel_name, indent = 4, sort_keys = True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.7099185,
                    "lng": -76.2145601
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.70838687010728,
                        "lng": -76.2132273201073
                    },
                    "southwest": {
                        "lat": -13.71108652989272,
                        "lng": -76.21592697989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "0f149222b85e950ade639a7307f5ce68e2c79697",
            "name": "Embassy Beach",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 856,
                    

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAMQsiGTKCjKTna3kagbx-L9fHFPGr0Mf088m1CN5GGb7Jf8ItbdSpznLyh_85kqrRFzCNnDdeRguFve8z2rpPxgmMKEl96lv_cGej3eVObMx0HBgC-5GOB_9Hps0rDf4w6vSN9xLyG4KZBAbrY1g1kmv2vPp1E2F1b2As26V273AsBLQxj_0bygHgv27_pdXtxXl_QxDNI26s3ezAfsH2KXLwSeOVjfx0-_3W-2yDWlReqq-G2vg30g9LV8nVRKeSl2l4dEK2eYrLnbIPT-VcEivUfN58gXZIiQL5zml-pEOi5y4t8PcuEJDYLtMA3ylhbAYrSDksbtqLWcNDpSPS0reChJLXcLVHlpxT2eC1wInLG0EDofKSi4A49jZe3XxDXk1ItzRSODANMqUD1lvBLESEO-v0owdWKuRN8CrPHn4sqMaFM4QHCtEyibtpg377ssKhLR0iB1q",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6335448,
                    "lng": -46.0303404
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.63216997010727,
                        "lng": -46.02897407010728
                    },
                    "southwest": {
                       

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 63.86499999999999,
                    "lng": 11.2572222
                },
                "viewport": {
                    "northeast": {
                        "lat": 63.86645212989272,
                        "lng": 11.25856267989272
                    },
                    "southwest": {
                        "lat": 63.86375247010728,
                        "lng": 11.25586302010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "J\u00e6gtvolden Fjordhotell",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1363,
                    "html_attributions": [
                      

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAA3JhLpyik4PDL4JHHaBnPQzWsSN1jW2iGYXQw3lZYN856VUQSkeoC8UerKLMhCPQyDsJS7m8etHc_T9_4P8d9aMmDMT1kLfGMquDYefmlma-RP6QXt1NkkegBe-J6M6jj-J3E629VnKD-PW0Uz_cR2TxQ0hLpsYOUjfO8ePyGwEkmpE0kJ6tlmAubrkYbIB7TfRQL2aCWcey0BGSfNigdP4ULHVKDrLBJ0FuyRc57USL6bZBylRDm35rNH57DPHzuosA3wrI_5x9Eo2jqcbu6BzCMj57BvzAniTY3YtTqNge_15m_h5hkPhEIjFoMvmx44PfAX-DzHqw8Ue0yf9z26gQGJ8u5mudzdchJQzBdhMF02PwIOqibAX6F_ZfldoKc619vbYmIRwDaTG_RWZEHwSELvGxRovJ5bErqMkLv3vWEIaFGZ9dqGZN0fq1d8DeWEGaIbHsBsg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7261401,
                    "lng": 139.4928025
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.72480357010728,
                        "lng": 139.4942367798927
                    },
                    "southwest": {
                        

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAACxgcXOobXjVqwQDTEJpXUWk4IptAfxORbfEO6VTMUVckS2yoFP_8t-kH9WWEWyx26xqhSMxJYZmsMIg39spgTSONdncaLRU-ibrPht5pSaW2DTcuMe8R968JBm3b6762HBeZG6zzN0zW088E19FCZlt7VT4WksHG2iPAhasvcSCFNz74rVf9mOog4zFBn8p7HwngSt4ukRoZnTQeeGtunAz-cU_vxNRnSJps8dkiBvavf1EfHnspX51O-NvCAoNVbKT2seXL4yOfWLuv0x_l_9UXpOK-8WEnXD1NCB7pRMN_i1hw4EeNlLrqhw9z_Ooxjpwo78Y6gjw4EHvxMMuD5lB4CEiRP26layEyvBmgT2YY4ulci2msvUtuCSypfAIrOd1F_aXSadL8QawJXa0OsgSENEjRk4esm1g9MTxtd2hVXQaFHJmys6proIwBDUbg6HA53PpoqsA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 47.1031883,
                    "lng": -119.3175825
                },
                "viewport": {
                    "northeast": {
                        "lat": 47.10448237989273,
                        "lng": -119.3158924701073
                    },
                    "southwest": {
                        

{
    "html_attributions": [],
    "next_page_token": "CrQCIwEAAL1stpF-Q_yqADe12UkEjaT6wejio_sOCp0xzNdGi2WMF_NiyIbnDkyvTDULgXwXvNqibnzq17ZoHn136_a8MLp7x_noLjcNi7jDwm1daEWASZOgjYy41BxNzqnptyvwHJ66Q1WQbo8Q7L2VqieSIeSOCYRiG8Ijq8bAjkjJZMKiZEK98lHWBMlnjnREg2Hs4cGNdWIXGYDn27iS5DvhNdr41hKaZXUrPjRatIImWCqruws5ojtcgwlM3gR1uG9r4AnADja4C6E2AfUSjqT1ZnThX_g1ijVuWnjPQS-yIdevKGk5JB-HVwLxpamuv2UhGSRfXrZYcH1Z9zD-l5tggj1FB6lPeoJs_5qPlSjFxJpzgCkONEZHWXsgQBMXNwwADaQQkt_UtQUyE9dEM1EszRASEHTKZvdwHhg9HSrl2tNfJo8aFBiY9T2MRpVjV-2zVxk4ovcrCnPb",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 42.8168371,
                    "lng": 93.520449
                },
                "viewport": {
                    "northeast": {
                        "lat": 42.81818932989272,
                        "lng": 93.52197687989273
                    },
                    "southwest": {
                        "lat

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.9096939,
                    "lng": 96.5038617
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.91113002989272,
                        "lng": 96.50508842989272
                    },
                    "southwest": {
                        "lat": 22.90843037010728,
                        "lng": 96.50238877010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "ec02eaccd6024d44e3e87fb57b2488111cee58e5",
            "name": "Mount Mogok Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 607,
                    "

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -30.711918,
                    "lng": 30.477708
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.71063502010728,
                        "lng": 30.47916612989273
                    },
                    "southwest": {
                        "lat": -30.71333467989272,
                        "lng": 30.47646647010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "Umthunzi Hotel & Conference",
            "photos": [
                {
                    "height": 1536,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/109202434759221014580\">Umthunzi Bo

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.045465,
                    "lng": 31.668
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.04411432010728,
                        "lng": 31.66934742989272
                    },
                    "southwest": {
                        "lat": -21.04681397989273,
                        "lng": 31.66664777010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "3cd5366ff4db23912e8272906453bbe352eea2af",
            "name": "The Nesbitt Arms",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 4912,
                    "ht

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>pisco</dd>\n<dt>Country</dt><dd>PE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>ponta do sol</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>straumen</dd>\n<dt>Country</dt><dd>NO</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>nador</dd>\n<dt>Country</dt><dd>MA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>mount isa</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>moses lake</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>hami</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>mogok</dd>\n<dt>Country</dt><dd>MM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>port shepstone</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>chir

In [12]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,-80))

markers = gmaps.marker_layer(locations)


# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…